**1.pip install**

In [2]:
# Install Google Earth Engine API
!pip install earthengine-api

# Import required libraries
import ee
import folium
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

# Authenticate Google Earth Engine
ee.Authenticate()

# Initialize Google Earth Engine with project ID
ee.Initialize(project='remote-sensing-454607')

print("Google Earth Engine Initialized Successfully!")


Google Earth Engine Initialized Successfully!


**2. Define AOI (Berambadi & Chennai)**

In [3]:
# Define the Berambadi Watershed AOI
berambadi_aoi = ee.Geometry.Polygon([
    [[76.6041, 11.7815], [76.5734, 11.7942], [76.5619, 11.7816], 
     [76.5562, 11.7565], [76.5737, 11.7507], [76.6056, 11.7481], 
     [76.6145, 11.7698], [76.6326, 11.7752], [76.6586, 11.7878], 
     [76.6536, 11.7968], [76.6294, 11.8103], [76.6041, 11.7815]]
])

# Define the Chennai AOI
chennai_aoi = ee.Geometry.Polygon([
    [[80.1914, 13.1575], [80.1914, 12.8380], 
     [80.3575, 12.8380], [80.3575, 13.1575]]
])

print("AOIs for Berambadi and Chennai Defined!")


AOIs for Berambadi and Chennai Defined!


**3. Retrieve Sentinel-2 NDVI (Berambadi & Chennai)**

In [4]:
# Function to compute NDVI from Sentinel-2
def get_ndvi(aoi, start_date, end_date):
    s2 = ee.ImageCollection('COPERNICUS/S2_SR') \
        .filterBounds(aoi) \
        .filterDate(start_date, end_date) \
        .select(['B4', 'B8']) \
        .median()

    ndvi = s2.expression(
        '(B8 - B4) / (B8 + B4)', {
            'B4': s2.select('B4'),
            'B8': s2.select('B8')
        }
    )
    return ndvi

# Get NDVI for Berambadi & Chennai
ndvi_berambadi = get_ndvi(berambadi_aoi, '2022-01-01', '2022-12-31')
ndvi_chennai = get_ndvi(chennai_aoi, '2022-01-01', '2022-12-31')

print("NDVI Computed for Both Locations!")


NDVI Computed for Both Locations!


/usr/local/lib/python3.10/dist-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for COPERNICUS/S2_SR! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR

  warnings.warn(warning, category=DeprecationWarning)


**4. Retrieve Sentinel-1 Backscatter (Berambadi & Chennai)**

In [5]:
# Function to get Sentinel-1 Backscatter
def get_backscatter(aoi, start_date, end_date):
    s1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filterBounds(aoi) \
        .filterDate(start_date, end_date) \
        .select(['VV', 'VH']) \
        .median()
    return s1

# Get Backscatter Data for Berambadi & Chennai
backscatter_berambadi = get_backscatter(berambadi_aoi, '2022-01-01', '2022-12-31')
backscatter_chennai = get_backscatter(chennai_aoi, '2022-01-01', '2022-12-31')

print("Backscatter Data Retrieved for Both Locations!")


Backscatter Data Retrieved for Both Locations!


**5. Export NDVI & Backscatter to Google Drive**

In [6]:
# Function to export image to Google Drive
def export_to_drive(image, description, region):
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=description,
        scale=30,
        region=region,
        fileFormat='GeoTIFF'
    )
    task.start()

# Export Berambadi Data
export_to_drive(ndvi_berambadi, 'NDVI_Berambadi', berambadi_aoi)
export_to_drive(backscatter_berambadi, 'Backscatter_Berambadi', berambadi_aoi)

# Export Chennai Data
export_to_drive(ndvi_chennai, 'NDVI_Chennai', chennai_aoi)
export_to_drive(backscatter_chennai, 'Backscatter_Chennai', chennai_aoi)

print("Exporting NDVI & Backscatter Data for Berambadi and Chennai...")


Exporting NDVI & Backscatter Data for Berambadi and Chennai...


**6.Visualizing AOI on Folium Map**

In [7]:
# Convert AOI to GeoJSON
berambadi_geojson = berambadi_aoi.getInfo()
chennai_geojson = chennai_aoi.getInfo()

# Create a Folium Map
map1 = folium.Map(location=[11.777968, 76.597909], zoom_start=10)
folium.GeoJson(data=berambadi_geojson, style_function=lambda x: {'fillColor': 'orange'}).add_to(map1)

map2 = folium.Map(location=[13.0827, 80.2707], zoom_start=10)
folium.GeoJson(data=chennai_geojson, style_function=lambda x: {'fillColor': 'blue'}).add_to(map2)

# Display Maps
print("Map of Berambadi:")
display(map1)

print("Map of Chennai:")
display(map2)


Map of Berambadi:


Map of Chennai:
